In [97]:
import requests
from fake_useragent import UserAgent
from bs4 import BeautifulSoup
import lxml

import time
from selenium import webdriver
from selenium.webdriver.common.by import By
from selenium.webdriver.common.keys import Keys

import pandas as pd

from datetime import datetime

#Отображение колонок и строк в VScode
pd.set_option('display.max_rows', None)
pd.set_option('display.max_columns', None)

In [98]:
profs_all = pd.read_csv('profs.txt', header=None, names=['profs_names'])
profs_test = pd.read_csv('profs_test.txt', header=None, names=['profs_names'])

In [99]:
vk_job = pd.DataFrame(columns=['link', 'name', 'location', 'company', 'date', 'description', 'date_of_download'])
df_test = pd.DataFrame(columns=['link', 'name', 'location', 'company', 'date', 'description', 'date_of_download'])

In [100]:
class VkJobParser:

    url = 'https://team.vk.company/vacancy/'
    
    def __init__(self, vac_type):
        '''
        Для инциалиазации задается название вакансии vac_type
        '''
        self.vac_type = vac_type
        self.browser = webdriver.Chrome()
        self.browser.get(self.url)
        self.browser.maximize_window()
        self.browser.delete_all_cookies()
        time.sleep(2)
        
    def stop(self):
        '''
        Выйти из Webdriver selenium
        '''
        self.browser.quit()
    
    def find_vacancies(self, profs):
        '''
        Функция для ввода названия вакансии (vac_type) в поисковую строку и клика клавиши Enter
        '''
        for i in range(0, len(profs)):

            top = self.browser.find_element(By.XPATH, '/html/body/div/section/div')
            self.browser.execute_script("return arguments[0].scrollIntoView(true);", top)

            input_button = self.browser.find_element(By.XPATH, '/html/body/div/div[1]/div[1]/div/form/div[1]/div[4]/div/div/div/div/input')

            input_button.send_keys(f"{profs['profs_names'][i]}")
            click_button = self.browser.find_element(By.CLASS_NAME, 'icon-form-search')
            click_button.click()
            time.sleep(5)

            # Прокрутка вниз до конца страницы
            page_height = self.browser.execute_script('return document.body.scrollHeight')

            while True:
                self.browser.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
                time.sleep(5)

                new_page_height = self.browser.execute_script('return document.body.scrollHeight')

                if new_page_height == page_height:
                    break
                else:
                    page_height = new_page_height
            try:
                vacs_bar = self.browser.find_element(By.XPATH, '/html/body/div/div[1]/div[2]/div/div')
                vacs = vacs_bar.find_elements(By.CLASS_NAME, 'result-item')
                vacs = [div for div in vacs if 'result-item' in str(div.get_attribute('class'))]
            
                for vac in vacs:
                    vac_info = {}
                    vac_info['link'] = vac.get_attribute('href')
                    vac_info['name'] = vac.find_element(By.CLASS_NAME, 'title-block').text
                    vac_info['location'] = vac.find_element(By.CLASS_NAME, 'result-item-place').text
                    vac_info['company'] = vac.find_element(By.CLASS_NAME, 'result-item-unit').text
                    vac_info['date_of_download'] = datetime.now()
                    df_test.loc[len(df_test)] = vac_info
                input_button.clear()
            except:
                input_button.clear()
                pass
            
        df_test.drop_duplicates(subset=['link'], keep='last')

    def find_description(self):
        for descr in range(len(df_test)):
            link = df_test.loc[descr, 'link']
            self.browser.get(link)
            self.browser.delete_all_cookies()
            time.sleep(3)

            df_test.loc[descr, 'description'] = self.browser.find_element(By.CLASS_NAME, 'section-cols').text

    def save_df(self):
        df_test.to_csv(f"all_{datetime.now()}.txt", index=False, sep=';')


In [101]:
parser = VkJobParser(profs_all)
parser.find_vacancies(profs_all)
parser.stop()

In [105]:
parser = VkJobParser(profs_all)
parser.find_description()
parser.stop()

In [107]:
df_test[:5]

,link,name,location,company,date,description,date_of_download
0,https://team.vk.company/vacancy/14451/,QA-инженер,"Москва, гибкий",ВКонтакте для бизнеса,NaN,"QA-инженер, Москва\nОткликнуться\nМы в поисках...",2023-10-08 16:20:49.216502
1,https://team.vk.company/vacancy/32523/,Дата-инженер,"Москва, гибкий",Дзен,NaN,"Дата-инженер, Москва\nОткликнуться\nО Дзене \n...",2023-10-08 16:20:49.259959
2,https://team.vk.company/vacancy/30929/,Аналитик данных,"Москва, комбинированный",VK,NaN,"Аналитик данных, Москва\nОткликнуться\nМы зани...",2023-10-08 16:20:49.303726
3,https://team.vk.company/vacancy/32888/,Data-инженер,"Москва, гибкий",Рекламные технологии и развитие СМБ,NaN,"Data-инженер, Москва\nОткликнуться\nИщем дата-...",2023-10-08 16:20:49.348671
4,https://team.vk.company/vacancy/31373/,DevOps-инженер,"Москва, гибкий",VK Tax Compliance,NaN,"DevOps-инженер, Москва\nОткликнуться\nМы — тех...",2023-10-08 16:20:49.397251


In [109]:
df_test.to_csv(f"all_vk.txt", index=False, sep=';')

In [70]:
for descr in range(len(df_test)):
    link = df_test.loc[descr, 'link']
    browser = webdriver.Chrome()
    browser.get(link)
    browser.maximize_window()
    browser.delete_all_cookies()
    time.sleep(2)

    df_test.loc[descr, 'description'] = browser.find_element(By.CLASS_NAME, 'section-cols').text

In [48]:
profs = pd.read_csv('profs.txt', sep=',', header=None, names=['profs_names', 'short_name'])
df = pd.DataFrame(columns=['link', 'name', 'location',
                           'company', 'date'])

with webdriver.Chrome() as browser:

    # заход на сайт
    browser.get('https://team.vk.company/vacancy/')
    browser.maximize_window()
    browser.delete_all_cookies()
    time.sleep(2)

    for i in range(0, len(profs_test)):
        input_button = browser.find_element(By.XPATH,
                                            '/html/body/div/div[1]/div[1]/div/form/div[1]/div[4]/div/div/div/div/input')

        input_button.send_keys(f"{profs_test['profs_names'][i]}")
        click_button = browser.find_element(By.XPATH,
                                            '/html/body/div/div[1]/div[1]/div/form/div[1]/div[4]/div/button')
        click_button.click()
        time.sleep(5)

        # Прокрутка вниз до конца страницы
        page_height = browser.execute_script('return document.body.scrollHeight')

        while True:
            browser.find_element(By.TAG_NAME, 'body').send_keys(Keys.END)
            time.sleep(2)

            new_page_height = browser.execute_script('return document.body.scrollHeight')

            if new_page_height == page_height:
                break
            else:
                page_height = new_page_height

        # Подсчет количества предложений
        
        vacs_bar = browser.find_element(By.XPATH, '/html/body/div/div[1]/div[2]/div/div')
        vacs = vacs_bar.find_elements(By.CLASS_NAME, 'result-item')

        vacs = [div for div in vacs if 'result-item' in str(div.get_attribute('class'))]
        print(f"Парсим вакансии по запросу: {profs_test['profs_names'][i]}")
        print(f"Количество: " + str(len(vacs)) + "\n")

        for vac in vacs:
            vac_info = {}
            vac_info['link'] = vac.get_attribute('href')
            vac_info['name'] = vac.find_element(By.CLASS_NAME, 'title-block').text
            vac_info['location'] = vac.find_element(By.CLASS_NAME, 'result-item-place').text
            vac_info['company'] = vac.find_element(By.CLASS_NAME, 'result-item-unit').text
            df.loc[len(df)] = vac_info

            

        input_button.clear()

        # Раскомментировать для загрузки данных в файлы по запросам
        # df.to_csv(f"loaded_data/{cur_year}/sber/{profs['short_name'][i]}-{cur_date}.txt", index=False, sep=';')
        # df = df.drop(df.index[0:])

#df = df.drop_duplicates()
#df.to_csv(f"all_{cur_date}.txt", index=False, sep=';')
#print("Общее количество вакансий: " + str(len(df)) + "\n")



Парсим вакансии по запросу: Инженер данных
Количество: 39

Парсим вакансии по запросу: Data engineer
Количество: 31



In [50]:
df

,link,name,location,company,date
0,https://team.vk.company/vacancy/14451/,QA-инженер,"Москва, гибкий",ВКонтакте для бизнеса,NaN
1,https://team.vk.company/vacancy/32523/,Дата-инженер,"Москва, гибкий",Дзен,NaN
2,https://team.vk.company/vacancy/30929/,Аналитик данных,"Москва, комбинированный",VK,NaN
3,https://team.vk.company/vacancy/32888/,Data-инженер,"Москва, гибкий",Рекламные технологии и развитие СМБ,NaN
4,https://team.vk.company/vacancy/31373/,DevOps-инженер,"Москва, гибкий",VK Tax Compliance,NaN
...,...,...,...,...,...
65,https://team.vk.company/vacancy/32800/,Data Scientist (реклама и медиа),"Москва, гибкий",VK Реклама,NaN
66,https://team.vk.company/vacancy/31884/,High performance engineer (Группа нейронных се...,"Москва, гибкий",Голосовые технологии (Маруся),NaN
67,https://team.vk.company/vacancy/33003/,Deep Learning Engineer (NLP),"Москва, гибкий",Голосовые технологии (Маруся),NaN
68,https://team.vk.company/vacancy/33085/,ML engineer,"Москва, гибкий",VK Реклама,NaN
